In [6]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce

from CurveInterpolator import GeneralCurveInterpolator
from CurveDataFetcher import CurveDataFetcher
from models.calibrate import (
    calibrate_ns_ols,
    calibrate_nss_ols,
    calibrate_mles_ols,
)
from utils.viz import plot_usts

import os
from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), ".env")
print(env_path)
config = dotenv_values(env_path)

import nest_asyncio
nest_asyncio.apply()

import matplotlib.pylab as pylab
params = {
    "legend.fontsize": "x-large",
    "figure.figsize": (20, 10),
    "axes.labelsize": "x-large",
    "axes.titlesize": "x-large",
    "xtick.labelsize": "x-large",
    "ytick.labelsize": "x-large",
}
pylab.rcParams.update(params)
sns.set(style="whitegrid", palette="dark")
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

%load_ext autoreload
%autoreload 2

c:\Users\chris\Curvy-CUSIPs\.env
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
curve_data_fetcher = CurveDataFetcher(use_ust_issue_date=True, fred_api_key=config["FRED_API_KEY"]) 

In [8]:
quote_type = "bid"
as_of_date = datetime(2024, 10, 1)

curve_set_df = curve_data_fetcher.build_curve_set(
    as_of_date=as_of_date,
    sorted=True,
    include_off_the_run_number=True,
    market_cols_to_return=[f"{quote_type}_price", f"{quote_type}_yield"],
    # calc_free_float=True,
    use_github=True,
)

curve_set_df

,cusip,security_type,auction_date,issue_date,maturity_date,time_to_maturity,int_rate,high_investment_rate,is_on_the_run,ust_label,security_term,original_security_term,bid_price,bid_yield,rank
0,912797LK1,Bill,2024-08-29,2024-09-03,2024-10-01,0.000000,NaN,5.263,False,5.263% Oct-24,4-Week,17-Week,NaN,NaN,17.0
1,912797LS4,Bill,2024-09-05,2024-09-10,2024-10-08,0.019178,NaN,5.171,False,5.171% Oct-24,4-Week,17-Week,99.907833,5.704622,16.0
2,912797LT2,Bill,2024-09-12,2024-09-17,2024-10-15,0.038356,NaN,5.053,False,5.053% Oct-24,4-Week,17-Week,99.816056,5.251256,15.0
3,912797LU9,Bill,2024-09-19,2024-09-24,2024-10-22,0.057534,NaN,4.783,False,4.783% Oct-24,4-Week,17-Week,99.722917,5.142662,14.0
4,912797LV7,Bill,2024-09-26,2024-10-01,2024-10-29,0.076712,NaN,4.783,True,4.783% Oct-24,4-Week,17-Week,99.629778,5.091610,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,912810TT5,Bond,2023-10-12,2023-10-16,2053-08-15,28.890411,4.125,NaN,False,4.125% Aug-53,29-Year 10-Month,30-Year,100.625000,4.087451,4.0
390,912810TV0,Bond,2024-01-11,2024-01-16,2053-11-15,29.142466,4.750,NaN,False,4.75% Nov-53,29-Year 10-Month,30-Year,111.593750,4.066550,3.0
391,912810TX6,Bond,2024-04-11,2024-04-15,2054-02-15,29.394521,4.250,NaN,False,4.25% Feb-54,29-Year 10-Month,30-Year,103.000000,4.073448,2.0
392,912810UA4,Bond,2024-07-11,2024-07-15,2054-05-15,29.638356,4.625,NaN,False,4.625% May-54,29-Year 10-Month,30-Year,109.593750,4.064522,1.0


In [239]:
otr_df = curve_set_df[curve_set_df["is_on_the_run"]].sort_values(by=["time_to_maturity"])
otr_fit_interpolator = GeneralCurveInterpolator(
    x=otr_df["time_to_maturity"].to_numpy(),
    y=otr_df[f"{quote_type}_yield"].to_numpy(),
)

In [241]:
# remove otr, olds, double olds, triple olds
curve_set_filtered_df = curve_set_df[
    (curve_set_df["rank"] != 0)
    & (curve_set_df["rank"] != 1)
    & (curve_set_df["rank"] != 2)
    & (curve_set_df["rank"] != 3)
]

# remove TBills
curve_set_filtered_df = curve_set_filtered_df[
    curve_set_filtered_df["security_type"] != "Bill"
]

# remove low free float bonds (< $5bn)
curve_set_filtered_df = curve_set_filtered_df[
    curve_set_filtered_df["free_float"] > 5000
]

# filter out bonds very close to maturity
curve_set_filtered_df = curve_set_filtered_df[
    curve_set_filtered_df["time_to_maturity"] > 30 / 360
]

# remove CTDs
curve_set_filtered_df = curve_set_filtered_df[
    ~curve_set_filtered_df["cusip"].isin(
        [
            curve_data_fetcher.ust_data_fetcher.cme_ust_label_to_cusip("4.625s 2026-09-15")["cusip"], # TU
            curve_data_fetcher.ust_data_fetcher.cme_ust_label_to_cusip("4.125s 2027-09-30")["cusip"], # Z3N
            curve_data_fetcher.ust_data_fetcher.cme_ust_label_to_cusip("4.25s 2029-02-28")["cusip"], # FV
            curve_data_fetcher.ust_data_fetcher.cme_ust_label_to_cusip("4.25s 2031-06-30")["cusip"], # TY
            curve_data_fetcher.ust_data_fetcher.cme_ust_label_to_cusip("4.375s 2034-05-15")["cusip"], # TN
            curve_data_fetcher.ust_data_fetcher.cme_ust_label_to_cusip("4.625s 2040-02-15")["cusip"], # US
            curve_data_fetcher.ust_data_fetcher.cme_ust_label_to_cusip("4.5s 2044-02-15")["cusip"], # TWE
            curve_data_fetcher.ust_data_fetcher.cme_ust_label_to_cusip("4.75s 2053-11-15")["cusip"], # UL
        ]
    )
]

curve_set_filtered_df = curve_set_filtered_df.sort_values(by=["time_to_maturity"])

filtered_fitted_interpolator = GeneralCurveInterpolator(
    x=curve_set_filtered_df["time_to_maturity"].to_numpy(),
    y=curve_set_filtered_df[f"{quote_type}_yield"].to_numpy(),
)

In [242]:
ns_func, status_ns = calibrate_ns_ols(
    curve_set_filtered_df["time_to_maturity"].to_numpy(),
    curve_set_filtered_df[f"{quote_type}_yield"].to_numpy(),
)
assert status_ns

nss_func, status_nss, _ = calibrate_nss_ols(
    curve_set_filtered_df["time_to_maturity"].to_numpy(),
    curve_set_filtered_df[f"{quote_type}_yield"].to_numpy(),
)
assert status_nss

mles_func, status_mles = calibrate_mles_ols(
    curve_set_filtered_df["time_to_maturity"].to_numpy(),
    curve_set_filtered_df[f"{quote_type}_yield"].to_numpy(),
    overnight_rate=5.31,
    N=9,
)

In [243]:
fitted_bspline = filtered_fitted_interpolator.b_spline_with_knots_interpolation(
    knots=[0.5, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 15, 20, 25],
    k=3,
    return_func=True,
)
fitted_unispline = filtered_fitted_interpolator.univariate_spline(s=1.75, return_func=True)

plot_usts(
    curve_set_df=curve_set_df,
    ttm_col="time_to_maturity",
    ytm_col=f"{quote_type}_yield",
    hover_data=[
        "issue_date",
        "maturity_date",
        "cusip",
        "label",
        f"{quote_type}_price",
        "free_float",
    ],
    ust_labels_highlighter=[("3.125% Nov-41", "red"), ("2.000% Nov-41", "blue")],
    zero_curves=[
        # (
        #     otr_fit_interpolator.pchip_interpolation(return_func=True),
        #     "PCHIP Intrp - OTR Fit",
        # ),
        # (
        #     fitted_bspline,
        #     "BSpline k=3 - Zero Filtered Fit",
        # ),
        (
            mles_func,
            "Zero, MLES"
        ),
    ],
    par_curves=[
        # (
        #     fitted_bspline,
        #     "BSpline k=3 - Par FF",
        # ),
        # (
        #     fitted_unispline,
        #     "Unispline s=1.75 - Par FF",
        # ),
        (
            mles_func,
            "Par, MLES",
            True
        ),
    ],
    # n_yr_fwd_curves=[
    #     (
    #         nss_func,
    #         1,
    #         "1yr Fwd, NSS",
    #     ) 
    # ]
    # impl_spot_n_yr_fwd_curves=[
    #     (fitted_bspline, 1, "Impl Spots, 1y Fwd"),
    # ],
    # impl_par_n_yr_fwd_curves=[
        # (fitted_bspline, 1, "Impl Par, 1y Fwd"),
    #     (
    #         mles_func,
    #         1,
    #         "1yr Fwd, MLES",
    #         True
    #     )
    # ],
)

c:\Users\chris\Curvy-CUSIPs\models\MLESM.py:47: RuntimeWarning:

divide by zero encountered in divide



In [254]:
nov_41s_df = curve_data_fetcher.fetch_spreads(
    ust_label_spread="3.125% Nov-41 / 2.000% Nov-41",
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 9, 26),
)

In [13]:
curve_sets_dict_df = curve_data_fetcher.fetch_historical_curve_sets(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 10, 1),
    fetch_soma_holdings=True,
    fetch_stripping_data=True,
    calc_free_float=True,
    # max_concurrent_tasks=256,
)

MERGING CURVE SET DFs: 100%|██████████| 242/242 [00:02<00:00, 108.62it/s]


In [40]:
curve_sets_dict_df[datetime(2023, 9, 5)]

,cusip,security_type,auction_date,issue_date,maturity_date,price_per100,allocation_pctage,avg_med_yield,bid_to_cover_ratio,comp_accepted,...,percentOutstanding,est_outstanding_amt,corpus_cusip_right,outstanding_amt,portion_unstripped_amt,portion_stripped_amt,reconstituted_amt,time_to_maturity,rank,free_float
0,912810FP8,Bond,2001-08-09,2001-08-15,2031-02-15,97.900,7.100000,5.472000,2.160000,null,...,0.506265,1.642765e+13,912803CK7,16427648.0,15447114.5,9.805335e+08,2300.0,7.452055,64,1.641835e+07
1,9128284R8,Note,2018-05-24,2018-05-31,2025-05-31,99.654192,33.680000,2.900000,2.620000,29974623800,...,0.360573,3.521160e+13,9128205F0,35211597.1,35094797.1,1.168000e+08,0.0,1.736986,63,3.519878e+07
2,91282CEB3,Note,2022-02-24,2022-02-28,2029-02-28,99.80427,63.560000,1.840000,2.360000,49985494800,...,0.146259,5.856583e+13,912821HM0,58565831.8,58565831.8,0.000000e+00,0.0,5.487671,18,5.855727e+07
3,912796ZN2,Bill,2023-06-26,2023-06-29,2023-12-28,97.363528,68.230000,null,2.730000,55868309500,...,0.090154,1.011131e+14,None,NaN,NaN,0.000000e+00,NaN,0.312329,8,1.011040e+08
4,912810SR0,Bond,2020-07-22,2020-07-31,2040-05-15,101.174893,33.470000,1.000000,2.430000,16999579200,...,0.332203,6.035664e+13,912803FQ1,60356639.2,59936109.2,4.205300e+08,7800.0,16.704110,13,6.033617e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,912810FB9,Bond,1998-02-12,1998-02-17,2027-11-15,null,92.000000,5.822000,null,null,...,0.646895,2.202134e+13,912803BM4,22021339.0,20682649.5,1.338690e+09,145000.0,4.197260,70,2.200575e+07
380,912810QB7,Bond,2009-07-09,2009-07-15,2039-05-15,99.104142,67.880000,4.230000,2.360000,10993568000,...,0.699990,3.877877e+13,912803DG5,38778773.0,37936033.7,8.427393e+08,5400.0,15.701370,57,3.875079e+07
381,912797HB6,Bill,2023-08-17,2023-08-22,2023-10-17,99.178667,92.960000,null,2.890000,68559598000,...,0.011956,1.173859e+14,None,NaN,NaN,0.000000e+00,NaN,0.115068,11,1.173845e+08
382,912828YB0,Note,2019-10-09,2019-10-15,2029-08-15,100.316063,59.010000,1.535000,2.430000,23996736500,...,0.508935,9.261867e+13,912821CR4,92618670.0,92588330.0,3.034000e+07,1000.0,5.947945,16,9.257150e+07


In [150]:
quote_type = "eod"
as_of_date = datetime(2024, 2, 5)

curve_set_df = curve_data_fetcher.build_curve_set(
    as_of_date=as_of_date,
    sorted=True,
    include_off_the_run_number=True,
    market_cols_to_return=[f"{quote_type}_price", f"{quote_type}_yield"],
    # calc_free_float=True,
    use_github=True,
)

curve_set_df

,cusip,security_type,auction_date,issue_date,maturity_date,time_to_maturity,int_rate,high_investment_rate,is_on_the_run,ust_label,security_term,original_security_term,eod_price,eod_yield,rank
0,912797JC2,Bill,2024-01-04,2024-01-09,2024-02-06,0.002740,NaN,5.400000,False,nan% Feb-24,4-Week,17-Week,100.000000,NaN,16.0
1,912797JD0,Bill,2024-01-11,2024-01-16,2024-02-13,0.021918,NaN,5.390000,False,nan% Feb-24,4-Week,17-Week,99.897528,5.463145,15.0
2,912797JE8,Bill,2024-01-18,2024-01-23,2024-02-20,0.041096,NaN,5.395000,False,nan% Feb-24,4-Week,17-Week,99.795056,5.465986,14.0
3,912797JF5,Bill,2024-01-25,2024-01-30,2024-02-27,0.060274,NaN,5.390000,True,nan% Feb-24,4-Week,17-Week,99.692583,5.468850,13.0
4,912797JG3,Bill,2024-01-04,2024-01-09,2024-03-05,0.079452,NaN,5.418000,False,nan% Mar-24,8-Week,17-Week,99.589333,5.421853,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,912810TL2,Bond,2023-01-12,2023-01-17,2052-11-15,28.797260,4.000,NaN,False,4.0% Nov-52,29-Year 10-Month,30-Year,93.875000,4.375659,4.0
382,912810TN8,Bond,2023-04-13,2023-04-17,2053-02-15,29.049315,3.625,NaN,False,3.625% Feb-53,29-Year 10-Month,30-Year,87.687500,4.378299,3.0
383,912810TR9,Bond,2023-07-13,2023-07-17,2053-05-15,29.293151,3.625,NaN,False,3.625% May-53,29-Year 10-Month,30-Year,87.750000,4.370108,2.0
384,912810TT5,Bond,2023-10-12,2023-10-16,2053-08-15,29.545205,4.125,NaN,False,4.125% Aug-53,29-Year 10-Month,30-Year,96.062500,4.363360,1.0
